<a href="https://colab.research.google.com/github/christopher-ell/Deep_Learning_Begin/blob/master/9_Custom_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Soucre: https://www.tensorflow.org/tutorials/eager/custom_layers

In [0]:
import tensorflow as tf

tf.enable_eager_execution()

**Layers: Common Sets of Useful Operations**

In [0]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of outputs dimensions / channels
## Create layer with 100 nodes so it has 100 outputs and however many inputs as
## required, since this is unspecified.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
## Create a layer with 10 nodes and so 10 outputs. The inputs are specified 
## with however many rows and 5 columns.
layer = tf.keras.layers.Dense(10, input_shape = (None, 5))

In [16]:
# To use a layer, simply call it.
## Load a tensor of zeros with 10 rows and 5 columns
layer(tf.zeros([10, 5]))

<tf.Tensor: id=665, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer by calling layer.variables. In this case a fully-connected layer
# will have variables for weights and biases.
## Output the parameters of all the layers 
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-2.5663665e-01, -5.8177799e-01,  1.8449670e-01, -2.9094586e-01,
         -4.1497099e-01, -5.5138874e-01, -1.1642277e-02,  4.5973164e-01,
         -3.2897383e-01,  2.0703471e-01],
        [-1.5895364e-01, -4.9259466e-01, -3.0101585e-01,  2.0438445e-01,
          1.1554521e-01, -2.6928544e-01, -2.7084821e-01, -5.1827544e-01,
          4.1160446e-01, -4.5114666e-01],
        [ 4.7656757e-01, -5.6489599e-01, -5.2124447e-01,  2.2124779e-01,
         -3.4935403e-01,  1.7513740e-01,  3.4350222e-01, -5.2504945e-01,
          1.8486959e-01, -1.5126956e-01],
        [-2.1642736e-01,  3.8950324e-02, -1.5366077e-04, -4.7530580e-01,
          3.9836526e-02,  1.8192589e-02,  2.8634286e-01, -1.4674303e-01,
          6.1877769e-01,  5.7627374e-01],
        [ 8.0519915e-04,  3.6500633e-02, -3.7201205e-01,  1.7403287e-01,
          6.8450272e-02,  2.1711373e-01,  5.8465141e-01, -5.1684564e-01,
          5.9309536e-01, -1.82319

In [5]:
## Show parameters of model broken down into kernel and bias parameters
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-2.5663665e-01, -5.8177799e-01,  1.8449670e-01, -2.9094586e-01,
         -4.1497099e-01, -5.5138874e-01, -1.1642277e-02,  4.5973164e-01,
         -3.2897383e-01,  2.0703471e-01],
        [-1.5895364e-01, -4.9259466e-01, -3.0101585e-01,  2.0438445e-01,
          1.1554521e-01, -2.6928544e-01, -2.7084821e-01, -5.1827544e-01,
          4.1160446e-01, -4.5114666e-01],
        [ 4.7656757e-01, -5.6489599e-01, -5.2124447e-01,  2.2124779e-01,
         -3.4935403e-01,  1.7513740e-01,  3.4350222e-01, -5.2504945e-01,
          1.8486959e-01, -1.5126956e-01],
        [-2.1642736e-01,  3.8950324e-02, -1.5366077e-04, -4.7530580e-01,
          3.9836526e-02,  1.8192589e-02,  2.8634286e-01, -1.4674303e-01,
          6.1877769e-01,  5.7627374e-01],
        [ 8.0519915e-04,  3.6500633e-02, -3.7201205e-01,  1.7403287e-01,
          6.8450272e-02,  2.1711373e-01,  5.8465141e-01, -5.1684564e-01,
          5.9309536e-01, -1.82319

**Implementing Custom Layers**

In [10]:
## Create own layer by extending the current layer
class MyDenseLayer(tf.keras.layers.Layer):
  ## Initialise new layer type by inheriting __init__ from original layer type 
  ## and then allowing specification of num_outputs
  def __init__(self, num_outputs):
    ## Inherit initiailisation from superclass
    super(MyDenseLayer, self).__init__()
    ## Make the number of inputs a class variable
    self.num_outputs = num_outputs
    
  ## Do initialisation when knowing the shape of the input tensors
  ## Build layer based on provided input shape and num_outputs
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape = [int(input_shape[-1]),
                                             self.num_outputs])
  
  ## Perform the forward computation of the custom layer
  def call(self, input):
    ## Multiply the model kernel parameters by the inputs
    return tf.matmul(input, self.kernel)
  
## Create a layer of 10 nodes (so 10 outputs) 
layer = MyDenseLayer(10)
## Print the layer after loading into it a 10 row and 5 column zeros matrix
print(layer(tf.zeros([10, 5])))
## Print the parameters of the model
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer_4/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.40735114, -0.6300019 , -0.25971937,  0.2380225 , -0.01751357,
        -0.60662204, -0.4422614 , -0.37311056, -0.22520524, -0.07270628],
       [-0.16349313, -0.5921618 ,  0.48648435, -0.1005854 ,  0.1952287 ,
         0.5262118 , -0.40402353,  0.50226647,  0.596348  ,  0.11827493],
       [ 0.37935656, -0.22028893, -0.21772596, -0.55010206,  0.16215414,
        -0.6152188 ,  0.1466257 , -0.06963807, -0.44268936,  0.11906266],
       [-0.6288277 , -0.3336715 ,  0.5596054 , -0.5340312 , -0.03087139,
         0.53430027, -0.30667317, 

**Models: Composing Layers**

In [12]:
## Create a layer like thing by composing existing layers by inheriting from 
## keras model class 
class ResnetIdentityBlock(tf.keras.Model):
  ## Initialise new class
  def __init__(self, kernel_size, filters):
    ## Inherit initialisation from keras model class
    super(ResnetIdentityBlock, self).__init__(name='')
    ## Unpack filters into three separate variables
    ## Filters give dimensiality of the output space
    filters1, filters2, filters3 = filters
    
    ## filter1 is the output from the convolution space
    ## (1, 1) is the kernel size that specifies the height and width of the 
    ## 2d convolution window.
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    ## Normalizes the activation of the previous layer. This one applies a 
    ## transformation to the activations layer making the mean activation close 
    ## to 0 and standard deviation close to 1.
    self.bn2a = tf.keras.layers.BatchNormalization()
    
    ## Create a convolutional layer with an output of filter2 and taking in 
    ## kernel sizes of kernel_size. 
    ## Also applies padding, but not sure what 'same' means
    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding = 'same')
    ## Normalises the activation of the previous layer. This one applies a 
    ## transformation to the activation layer making the mean activation close
    ## to 0 and standard deviation close to 1.
    self.bn2b = tf.keras.layers.BatchNormalization()
    
    ## Create a convolutional layer with an output of filter 3and taking in a 
    ## kernel size of 1 row by 1 column
    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    ## Normalises the activation of the previous layer. This one applies a 
    ## transformation to the activation layer making the mean activation close 
    ## to 0 and standard deviation close to 1
    self.bn2c = tf.keras.layers.BatchNormalization()
    
  ## Does a forward pass of the model taking the input data through the 
  ## convolutional neural network
  def call(self, input_tensor, training = False):
    ## Run the custom convolutional conv2a specified in __init__ specified above
    x = self.conv2a(input_tensor)
    ## Run the above custom layer normalisation specified in __init__
    x = self.bn2a(x, training = training)
    ## Put the output from the activation layer through a non-linear relu 
    ## function
    x = tf.nn.relu(x)
    
    ## Run the custom convolution function conv2b specified in __init__ above 
    x = self.conv2b(x)
    ## Run the above custom normalisation specified in __init__
    x = self.bn2b(x, training = training)
    ## Put the output from the activation layer through a non-linear relu
    ## function
    x = tf.nn.relu(x)
    
    ## Run the custom concolution function conv2c specified in __init__ above
    x = self.conv2c(x)
    ## Run the above normalisation specified in the __init__ above
    x = self.bn2c(x, training = training)
    
    ## Add the input tensor to the output (not sure why???) 
    x += input_tensor
    ## Run the output a final time through a non-linear relu function
    return tf.nn.relu(x)
  
## Create an instance of the ResnetIdnetityBlock class
block = ResnetIdentityBlock(1, [1, 2, 3])
## Run a tensor of zeros through the class instance
print(block(tf.zeros([1, 2, 3, 3])))
## Output the tensor being run through the model
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0', 'resnet_identity_block/batch_normalization/moving_mean:0', 'resnet_identity_block/batch_normalization/moving_variance:0', 'resnet_identity_block/batch_normalization_1/moving_mean:0', 'resnet_identity_block/batch_normalization_1/moving_variance:0', 'resnet_identity_block/batch_normalization_2

In [14]:
## Create a more staright forward model by calling the layers one after another 
## to be run sequentially

## The first layer is a convolutional layer with an output of one and blocks of
## 1 row by 1 column
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, [1, 1]),
## Normalise the previous layer to mean of 0 and std of 1
                             tf.keras.layers.BatchNormalization(),
## Run a second convolutional layer with an output of 2 dimensions and blocks of
## an input of 1 and padding.
                             tf.keras.layers.Conv2D(2, 1,
                                                   padding = 'same'),
## Normalise the previous layer to a mean of 0 and std of 1
                             tf.keras.layers.BatchNormalization(),
## Run a third convolutional layer of with output of 3 dimensions and a input of
## a 1 x 1 tensor
                             tf.keras.layers.Conv2D(3, (1, 1)), 
## Normalise the activation layer so the mean is 0 and std is 1
                             tf.keras.layers.BatchNormalization()])
## Run the above model with a tensor of 0's as input
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=628, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>